In [13]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from collections import defaultdict, Counter

In [2]:
df = pd.read_csv("../data/analysis_data/top1000_merged_profession_frequency.csv", index_col=None)

In [3]:
df.shape

(1000, 69)

In [4]:
df.head()

,profession_merge,1950,1951,1952,1953,1954,1955,1956,1957,1958,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,academician,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,7.054062e-07,3.495716e-07,3.271536e-07,...,1.187437e-07,2.477584e-08,2.353053e-07,7.760025e-08,3.795555e-08,9.127312e-08,1.818845e-08,1.785370e-08,1.797955e-08,0.000000e+00
1,accountant,0.000014,2.912382e-06,0.000008,2.953883e-06,0.000004,2.389870e-06,1.199190e-05,1.118629e-05,5.234458e-06,...,5.670011e-06,5.475462e-06,4.684715e-06,6.246820e-06,5.750266e-06,5.348605e-06,4.947257e-06,4.641961e-06,6.724351e-06,4.120042e-06
2,acolyte,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,6.543072e-07,...,2.374874e-07,2.973101e-07,4.706107e-07,2.716009e-07,2.277333e-07,2.008009e-07,7.639147e-07,3.213666e-07,3.416114e-07,6.699255e-07
3,acrobat,0.000005,4.853971e-07,0.000002,9.846275e-07,0.000002,6.828201e-07,1.410812e-06,6.292289e-06,3.271536e-06,...,8.015198e-07,3.716377e-07,6.417418e-07,4.656015e-07,5.124000e-07,3.650925e-07,6.184072e-07,7.320016e-07,4.854478e-07,7.704143e-07
4,actor,0.000030,2.329906e-05,0.000041,3.544659e-05,0.000033,1.707050e-05,2.116218e-05,3.950159e-05,2.878952e-05,...,2.977498e-05,3.017698e-05,2.708150e-05,2.692729e-05,2.856155e-05,2.893358e-05,2.731905e-05,2.754826e-05,3.315429e-05,2.003077e-05


In [5]:
df["spearman"] = df.apply(lambda x: x[1:].sum(), axis=1)

In [9]:
def trend(x):
    res = spearmanr(x[1:-1], np.arange(1950, 2018))
    if res.pvalue < 0.05:
        return res.correlation > 0
    else:
        return None

In [10]:
df["spearman"] = df.apply(trend, axis=1)

In [14]:
Counter(df["spearman"])

Counter({None: 348, True: 452, False: 200})

In [15]:
df.to_csv("../data/analysis_data/top1000_merged_profession_frequency_with_trend.csv", index=False)

In [16]:
sentiment_df = pd.read_csv("../data/analysis_data/top500_merged_profession_sentiment.csv", index_col=None)

In [17]:
sentiment_df.shape

(102000, 4)

In [18]:
sentiment_df.head()

,profession,year,sentiment,imdb
0,doctor,1950,-1,31
1,doctor,1950,0,1398
2,doctor,1950,1,112
3,doctor,1951,-1,19
4,doctor,1951,0,1209


In [19]:
records = []

for profession, pdf in sentiment_df.groupby("profession"):
    record = [profession]
    for year, ydf in pdf.groupby("year"):
        values = ydf["imdb"].values
        pos = values[2]
        neg = values[0]
        sentiment = pos / (pos + neg + 1e-23)
        record.append(sentiment)
    records.append(record)

In [20]:
sdf = pd.DataFrame(records, columns=["profession"] + np.arange(1950, 2018).astype(str).tolist())

In [22]:
sdf.to_csv("../data/analysis_data/top500_merged_profession_sentiment_trend.csv", index=False)